### **Protein Identification Interactive Model**
This Jupyter notebook allows for experimentation with the protein identification model! The last Python module shows a demonstration of how the model works, and the other modules are there to prepare the model.

**Importing Libraries**

In [7]:
# import library dependencies, tested on python 3.9.6
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow import keras

**Loading the Model**<br>
I load the model from a set directory and print a summary to visualize how it works.

In [8]:
# load the pre-trained model from the appropriate file path
model = tf.keras.models.load_model('saved_model/atlas/atlas_model.keras')
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_4 (Sequential)       │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_5 (Rescaling)         │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 100, 100, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 50, 50, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 50, 50, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 25, 25, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 25, 25, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ outputs (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,834,347 (14.63 MB)

 Trainable params: 1,278,115 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,556,232 (9.75 MB)

**Model Variables**<br>
Here, I take key variables (for example, img_height, and batch_size) to make sure the image identification model can identify images. I also remake the training and validation datasets to find the classes.

In [19]:
# redefine values from the model
data_dir = "imgs/atlas-imgs/"
img_height = img_width = 100
batch_size = 32

# remake the training dataset
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# remake the validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# find the class names
class_names = train_ds.class_names



Found 3000 files belonging to 3 classes.
Using 2400 files for training.
Found 3000 files belonging to 3 classes.
Using 2400 files for training.


**Image Identification**<br>
Here, the model takes in a filepath and identifies the top three likeliest possibilities. Feel free to tweak the URL to determine the model's accuracy!

In [20]:
# take an image to use for testing
path = "saved_model/atlas/coagulation_model_slice_094.png"

img = image.load_img(
    path, target_size=(img_height, img_width)
)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])
top_three = np.array(score).argsort()[-3:][::-1]
numpy_array = score.numpy()

# convert the folder names into English words then print the three likeliest probabilities
for i in top_three:
    words = class_names[i].split("_")
    name = " ".join([word.capitalize() for word in words])
    print(
        "This image likely belongs to {} with {:.2f}% confidence."
        .format(name, 100 * numpy_array[i])
    )

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
This image likely belongs to Rif1 with 100.00% confidence.
This image likely belongs to Coagulation with 0.00% confidence.
This image likely belongs to Bltp2 with 0.00% confidence.
